## Imports

In [46]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
from torchvision.transforms import Compose, ToTensor, Normalize, Resize, ToPILImage, CenterCrop, RandomResizedCrop
from torchvision.datasets import ImageFolder
from torchvision.models import alexnet, resnet18, inception_v3

from torchvision.models.alexnet import AlexNet_Weights
from torchvision.models.inception import Inception_V3_Weights
from torchvision.models.resnet import ResNet18_Weights

data_path = "../data/"

## Data preparation

In [47]:
from download_rps import download_rps

train_data_path = data_path + "rps"
val_data_path = data_path + "rps-test-set"
download_rps(data_path)

rps folder already exists!
rps-test-set folder already exists!


In [48]:
# ImageNet statistics
normalizer = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transformer = Compose([Resize(256), CenterCrop(224), ToTensor(), normalizer])

#transformer = ResNet18_Weights.IMAGENET1K_V1.transforms

train_data = ImageFolder(root=train_data_path, transform=transformer)
val_data = ImageFolder(root=val_data_path, transform=transformer)

# Builds a loader of each set
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=16)

## Feature extraction

### Model configuration

In [49]:
def freeze_model(model):
    for parameter in model.parameters():
        parameter.requires_grad = False

# Set the seed
torch.manual_seed(42)

# Load the model
resnet = resnet18(weights=ResNet18_Weights.DEFAULT)

# Change the top layer to Identity
resnet.fc = nn.Identity()
# Freeze the model
freeze_model(resnet)

### Preprocess data

In [50]:
def preprocess_dataset(model, dataset):
    features = torch.Tensor()
    labels = torch.Tensor()
    for x, y in dataset:
        model.eval()
        features = torch.cat([features, model(x)])
        labels = torch.cat([labels, y])

    dataset = TensorDataset(features, labels)
    return dataset

# Preprocess the data
train_preproc = preprocess_dataset(resnet, train_loader)
val_preproc = preprocess_dataset(resnet, val_loader)

### Save features

In [51]:
train_preproc_path = data_path + "train_preproc.pth"
val_preproc_path = data_path + "val_preproc.pth"
torch.save(train_preproc.tensors, train_preproc_path)
torch.save(val_preproc.tensors, val_preproc_path)

### Load features

In [52]:
train_preproc_path = data_path + "train_preproc.pth"
val_preproc_path = data_path + "val_preproc.pth"
train_preproc_data = TensorDataset(*torch.load(train_preproc_path))
val_preproc_data = TensorDataset(*torch.load(val_preproc_path))
train_preproc_loader = DataLoader(train_preproc_data, batch_size=16, shuffle=True)
val_preproc_loader = DataLoader(val_preproc_data, batch_size=16)

## Top model

### Model configuration

In [53]:
torch.manual_seed(42)
top_model = nn.Sequential(nn.Linear(512, 3))
multi_loss_fn = nn.CrossEntropyLoss(reduction='mean')
optimizer_top = optim.Adam(top_model.parameters(), lr=3e-4)

### Model training and evaluation

In [60]:
def train(model, train_loader, val_loader, loss_fn, optimizer, n_epochs):
    for epoch in range(n_epochs):
        model.train()
        for x, y in train_loader:
            optimizer.zero_grad()
            y_pred = model(x)
            loss = loss_fn(y_pred, y.long())
            loss.backward()
            optimizer.step()

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for x, y in val_loader:
                y_pred = model(x)
                _, predicted = torch.max(y_pred, 1)
                total += y.size(0)
                correct += (predicted == y).sum().item()

        print(f"Epoch {epoch+1}/{n_epochs} - Loss: {loss.item()} - Accuracy: {correct}/{total}")

train(top_model, train_preproc_loader, val_preproc_loader, multi_loss_fn, optimizer_top, 10)

Epoch 1/10 - Loss: 0.017219942063093185 - Accuracy: 307/372
Epoch 2/10 - Loss: 0.0028221302200108767 - Accuracy: 310/372
Epoch 3/10 - Loss: 0.0025548457633703947 - Accuracy: 298/372
Epoch 4/10 - Loss: 0.004645944572985172 - Accuracy: 309/372
Epoch 5/10 - Loss: 0.003532351227477193 - Accuracy: 306/372
Epoch 6/10 - Loss: 0.0018686791881918907 - Accuracy: 302/372
Epoch 7/10 - Loss: 0.0041360086761415005 - Accuracy: 306/372
Epoch 8/10 - Loss: 0.0027292410377413034 - Accuracy: 305/372
Epoch 9/10 - Loss: 0.005559865385293961 - Accuracy: 307/372
Epoch 10/10 - Loss: 0.0016467131208628416 - Accuracy: 301/372


## Using the original dataset

### Reattach the top model

In [61]:
def evaluate(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in data_loader:
            y_pred = model(x)
            _, predicted = torch.max(y_pred, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()

    print(f"Accuracy: {correct}/{total}")

resnet.fc = top_model
evaluate(resnet, val_loader)




Accuracy: 301/372
